In [1]:
from mbientlab.metawear import MetaWear, libmetawear, parse_value
import mbientlab.metawear.cbindings as cbindings
from ctypes import Structure, c_void_p, cast, POINTER, c_int, c_float
from threading import Event
from time import sleep
import pandas as pd

In [2]:
# MAC address of the MetaWear device
device_address = 'D4:06:BD:85:8F:23'

# Create and connect the device
device = MetaWear(device_address)
device.connect()

In [3]:
timestamps = []
x_accs = []
y_accs = []
z_accs = []
x_gyros = []
y_gyros = []
z_gyros = []

def data_handler(context, data):
    values = parse_value(data, n_elem = 2)
    timestamp = int(data.contents.epoch)
    print("timestamp: %.1f, acc: (%.4f,%.4f,%.4f), gyro; (%.4f,%.4f,%.4f)" % (timestamp, values[0].x, values[0].y, values[0].z, values[1].x, values[1].y, values[1].z))
    timestamps.append(timestamp)
    x_accs.append(values[0].x)
    y_accs.append(values[0].y)
    z_accs.append(values[0].z)
    x_gyros.append(values[1].x)
    y_gyros.append(values[1].y)
    z_gyros.append(values[1].z)


In [4]:
e = Event()
callback = cbindings.FnVoid_VoidP_DataP(data_handler)

# Callback for processor creation
def processor_created(context, pointer):
    global processor
    processor = pointer
    e.set()

# Function pointer
fn_wrapper = cbindings.FnVoid_VoidP_VoidP(processor_created)

# Acc and gyro signals
acc = libmetawear.mbl_mw_acc_get_acceleration_data_signal(device.board)
gyro = libmetawear.mbl_mw_gyro_bmi160_get_rotation_data_signal(device.board)

# Fuse the signals together
signals = (c_void_p * 1)()
signals[0] = gyro
libmetawear.mbl_mw_dataprocessor_fuser_create(acc, signals, 1, None, fn_wrapper)
e.wait()

# Subscribe to it
libmetawear.mbl_mw_datasignal_subscribe(processor, None, callback)

# Start the gyro and acc
libmetawear.mbl_mw_gyro_bmi160_enable_rotation_sampling(device.board)
libmetawear.mbl_mw_acc_enable_acceleration_sampling(device.board)
libmetawear.mbl_mw_gyro_bmi160_start(device.board)
libmetawear.mbl_mw_acc_start(device.board)

# Stream data for a duration (e.g., 30 seconds)
sleep(5)

# Stop and clean up
libmetawear.mbl_mw_acc_stop(device.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(device.board)
libmetawear.mbl_mw_datasignal_unsubscribe(processor)
libmetawear.mbl_mw_debug_disconnect(device.board)
print("Disconnected and cleaned up")

timestamp: 1714042957782.0, acc: (0.3210,-0.8368,0.3152), gyro; (4.2073,3.2317,2.5000)
timestamp: 1714042957784.0, acc: (0.3571,-0.8550,0.3428), gyro; (4.9390,4.2073,5.6098)
timestamp: 1714042957784.0, acc: (0.3801,-0.8644,0.3412), gyro; (4.6341,4.5732,5.6098)
timestamp: 1714042957784.0, acc: (0.3569,-0.8534,0.3416), gyro; (5.5488,8.7195,5.1829)
timestamp: 1714042957784.0, acc: (0.3425,-0.8441,0.3523), gyro; (5.2439,7.0732,4.8780)
timestamp: 1714042957785.0, acc: (0.3254,-0.8470,0.3425), gyro; (4.7561,6.7683,5.4268)
timestamp: 1714042957842.0, acc: (0.3303,-0.8629,0.3530), gyro; (6.3415,9.2683,8.5976)
timestamp: 1714042957842.0, acc: (0.3401,-0.8737,0.3512), gyro; (5.7317,7.6220,8.6585)
timestamp: 1714042957843.0, acc: (0.3349,-0.8789,0.3504), gyro; (3.7195,5.8537,5.3659)
timestamp: 1714042957843.0, acc: (0.3258,-0.8802,0.3483), gyro; (3.3537,5.1829,4.1463)
timestamp: 1714042957844.0, acc: (0.3282,-0.8828,0.3542), gyro; (3.7195,5.3049,4.2073)
timestamp: 1714042957962.0, acc: (0.3488,-0

In [7]:
device.on_disconnect = lambda status: print ("we are disconnected!")
device.disconnect()

we are disconnected!


In [8]:


#data_array = np.array([timestamps, x_accs, y_accs, z_accs, x_gyros, y_gyros, z_gyros])
data_df = pd.DataFrame(columns=['time', 'x_acc', 'y_acc', 'z_acc', 'x_gyro', 'y_gyro', 'z_gyro'])

data_df['time'] = timestamps
data_df['x_acc'] = x_accs
data_df['y_acc'] = y_accs
data_df['z_acc'] = z_accs
data_df['x_gyro'] = x_gyros
data_df['y_gyro'] = y_gyros
data_df['z_gyro'] = z_gyros

,time,x_acc,y_acc,z_acc,x_gyro,y_gyro,z_gyro
0,1714042957782,0.321045,-0.836792,0.315247,4.207317,3.231707,2.500000
1,1714042957784,0.357117,-0.854980,0.342834,4.939024,4.207317,5.609756
2,1714042957784,0.380127,-0.864380,0.341248,4.634146,4.573171,5.609756
3,1714042957784,0.356934,-0.853394,0.341553,5.548780,8.719512,5.182927
4,1714042957784,0.342529,-0.844116,0.352295,5.243903,7.073171,4.878049
5,1714042957785,0.325439,-0.846985,0.342529,4.756098,6.768293,5.426829
6,1714042957842,0.330322,-0.862854,0.353027,6.341464,9.268292,8.597561
7,1714042957842,0.340149,-0.873657,0.351196,5.731708,7.621952,8.658537
8,1714042957843,0.334900,-0.878906,0.350403,3.719512,5.853659,5.365854
9,1714042957843,0.325806,-0.880249,0.348328,3.353659,5.182927,4.146341
